In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
from bs4 import Comment
import requests
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
teams = ['ATL', 'BRK', 'BOS', 'CHO', 'CHI', 
         'CLE', 'DAL', 'DEN', 'DET', 'GSW', 
         'HOU', 'IND', 'LAC', 'LAL', 'MEM', 
         'MIA', 'MIL', 'MIN', 'NOP', 'NYK', 
         'OKC', 'ORL', 'PHI', 'PHO', 'POR', 
         'SAC', 'SAS', 'TOR', 'UTA', 'WAS']

for team in teams:
    team_url = 'https://www.basketball-reference.com/teams/'+team+'/2020.html'
    print(team+':')
    print(pd.read_html(team_url)[0])
    

ATL:
    No.            Player Pos    Ht   Wt          Birth Date Unnamed: 6 Exp  \
0    95   DeAndre' Bembry  SG   6-5  210        July 4, 1994         us   3   
1     4     Charlie Brown  SG   6-6  199    February 2, 1997         us   R   
2    15      Vince Carter  PF   6-6  220    January 26, 1977         us  21   
3    20      John Collins  PF   6-9  235  September 23, 1997         us   2   
4    33      Allen Crabbe  SG   6-5  212       April 9, 1992         us   6   
5    14    Dewayne Dedmon   C   7-0  245     August 12, 1989         us   6   
6    24    Bruno Fernando   C   6-9  240     August 15, 1998         ao   R   
7     0   Brandon Goodwin  PG   6-0  180     October 2, 1995         us   1   
8     2    Treveon Graham  SF   6-5  219    October 28, 1993         us   3   
9     3     Kevin Huerter  SG   6-7  190     August 27, 1998         us   1   
10   12   De'Andre Hunter  SF   6-8  225    December 2, 1997         us   R   
11   30      Damian Jones   C  6-11  245       

    No.              Player Pos    Ht   Wt         Birth Date Unnamed: 6 Exp  \
0    51    Ryan Arcidiacono  PG   6-3  195     March 26, 1994         us   2   
1    34      Wendell Carter   C  6-10  270     April 16, 1999         us   1   
2    32           Kris Dunn  PG   6-3  205     March 18, 1994         us   3   
3     6   Cristiano Felício   C  6-11  270       July 7, 1992         br   4   
4    12      Daniel Gafford   C  6-10  234    October 1, 1998         us   R   
5     3  Shaquille Harrison  SG   6-7  190    October 6, 1993         us   2   
6    15  Chandler Hutchison  SF   6-6  210     April 26, 1996         us   1   
7     2         Luke Kornet   C   7-2  250      July 15, 1995         us   2   
8     8         Zach LaVine  SG   6-5  200     March 10, 1995         us   5   
9    24     Lauri Markkanen  PF   7-0  240       May 22, 1997         fi   2   
10   20         Adam Mokoka  SF   6-4  190      July 18, 1998         fr   R   
11   22         Otto Porter  SF   6-8  1

      No.                 Player Pos    Ht   Wt          Birth Date  \
0      18            Jordan Bone  PG   6-2  180    November 5, 1997   
1       6            Bruce Brown  PG   6-4  202     August 15, 1996   
2      45        Sekou Doumbouya  PF   6-8  230   December 23, 2000   
3       0         Andre Drummond   C  6-10  279     August 10, 1993   
4      12            Tim Frazier  PG   6-0  170    November 1, 1990   
5       9      Langston Galloway  SG   6-1  200    December 9, 1991   
6      23          Blake Griffin  PF  6-10  250      March 16, 1989   
7      42             Donta Hall  PF   6-9  232      August 7, 1997   
8      31            John Henson   C   6-9  219   December 28, 1990   
9       1         Reggie Jackson  PG   6-2  208      April 16, 1990   
10      5           Luke Kennard  SG   6-5  206       June 24, 1996   
11     14             Louis King  SF   6-7  205       April 6, 1999   
12     20         Brandon Knight  PG   6-2  182    December 2, 1991   
13    

      No.             Player Pos    Ht   Wt         Birth Date Unnamed: 6 Exp  \
0      21   Patrick Beverley  PG   6-1  180      July 12, 1988         us   7   
1       7        Amir Coffey  SG   6-7  210      June 17, 1997         us   R   
2      13        Paul George  SF   6-8  220        May 2, 1990         us   9   
3       4     JaMychal Green  PF   6-8  227      June 21, 1990         us   5   
4   8, 11   Maurice Harkless  PF   6-7  220       May 11, 1993         us   7   
5       5   Montrezl Harrell   C   6-7  240   January 26, 1994         us   4   
6       1     Reggie Jackson  SG   6-2  208     April 16, 1990         it   8   
7      25  Mfiondu Kabengele   C   6-9  250    August 14, 1997         ca   R   
8       2      Kawhi Leonard  SF   6-7  225      June 29, 1991         us   8   
9      14       Terance Mann  SF   6-5  215   October 18, 1996         us   R   
10     19    Rodney McGruder  SG   6-4  205      July 29, 1991         us   3   
11     31      Marcus Morris

    No.                  Player Pos    Ht   Wt         Birth Date Unnamed: 6  \
0    34   Giannis Antetokounmpo  PF  6-11  242   December 6, 1994         gr   
1    43  Thanasis Antetokounmpo  SF   6-6  219      July 18, 1992         gr   
2    17           Dragan Bender   C   7-0  225  November 17, 1997         ba   
3     6            Eric Bledsoe  PG   6-1  214   December 9, 1989         us   
4    23          Sterling Brown  SF   6-5  219  February 10, 1995         us   
5    24         Pat Connaughton  SG   6-5  209    January 6, 1993         us   
6     0        Donte DiVincenzo  SG   6-4  203   January 31, 1997         us   
7     3             George Hill  PG   6-4  188        May 4, 1986         us   
8     7          Ersan İlyasova  PF   6-9  235       May 15, 1987         tr   
9    26             Kyle Korver  SF   6-7  212     March 17, 1981         us   
10   11             Brook Lopez   C   7-0  282      April 1, 1988         us   
11   42             Robin Lopez   C   7-

    No.                   Player Pos    Ht   Wt          Birth Date  \
0    12             Steven Adams   C  6-11  265       July 20, 1993   
1     7            Darius Bazley  PF   6-8  208       June 12, 2000   
2    30            Deonte Burton  SF   6-4  240    January 31, 1994   
3     6           Hamidou Diallo  SF   6-5  202       July 31, 1998   
4     5            Luguentz Dort  SF   6-3  215      April 19, 1999   
5    23        Terrance Ferguson  SF   6-6  190        May 17, 1998   
6     8         Danilo Gallinari  PF  6-10  233      August 8, 1988   
7     2  Shai Gilgeous-Alexander  SG   6-6  180       July 12, 1998   
8    14               Devon Hall  PG   6-2  215        July 7, 1995   
9    15             Kevin Hervey  PF   6-9  230        July 9, 1996   
10   33             Mike Muscala   C  6-10  240        July 1, 1991   
11   11              Abdel Nader  SF   6-5  225  September 25, 1993   
12    9             Nerlens Noel   C  6-11  220      April 10, 1994   
13   1

    No.            Player Pos    Ht   Wt          Birth Date Unnamed: 6 Exp  \
0    10      Jaylen Adams  PG   6-0  225         May 4, 1996         us   1   
1     0   Carmelo Anthony  PF   6-7  238        May 29, 1984         us  16   
2     8      Trevor Ariza  SF   6-8  215       June 30, 1985         us  15   
3    24     Kent Bazemore  SF   6-4  195        July 1, 1989         us   7   
4     4       Moses Brown   C   7-2  245    October 13, 1999         us   R   
5    33      Zach Collins  PF  6-11  250   November 19, 1997         us   2   
6    35    Wenyen Gabriel  PF   6-9  205      March 26, 1997         sd   R   
7    44     Mario Hezonja  PF   6-8  220   February 25, 1995         hr   4   
8     6      Jaylen Hoard  PF   6-8  216      March 30, 1999         fr   R   
9     5       Rodney Hood  SF   6-8  208    October 20, 1992         us   5   
10   17   Skal Labissière   C  6-10  235      March 18, 1996         ht   3   
11    0    Damian Lillard  PG   6-2  195       July 

       No.                 Player Pos    Ht   Wt          Birth Date  \
0       44       Bojan Bogdanović  SF   6-7  226      April 18, 1989   
1       13           Tony Bradley   C  6-10  248     January 8, 1998   
2        5       Jarrell Brantley  PF   6-5  250        June 7, 1996   
3       00        Jordan Clarkson  SG   6-4  194        June 7, 1992   
4       10            Mike Conley  PG   6-1  175    October 11, 1987   
5       17               Ed Davis   C   6-9  218        June 5, 1989   
6       11             Dante Exum  SG   6-5  214       July 13, 1995   
7       27            Rudy Gobert   C   7-1  258       June 26, 1992   
8       22             Jeff Green  PF   6-8  235     August 28, 1986   
9        2             Joe Ingles  PF   6-8  220     October 2, 1987   
10      15           Stanton Kidd  SG   6-6  215      March 18, 1992   
11      45       Donovan Mitchell  SG   6-1  215   September 7, 1996   
12      16           Juwan Morgan  PF   6-7  232      April 17, 

In [3]:
# Try to scrape the site using the URL
url = 'https://www.basketball-reference.com/teams/LAL/2020.html'

# Retrieve page with the requests module
html = requests.get(url)

# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html.text, 'html.parser')

# Get article title and summary paragraph

ballers = {'Name': [], 'Page': [], 'Photo': []}

for x in range(1,50):
    try:
        ballers['Name'].append(soup.find_all('tr')[x].a.text)
        ballers['Page'].append('https://www.basketball-reference.com/' + soup.find_all('tr')[x].a['href'])
    except:
        break
        
ballers


{'Name': ['Kostas Antetokounmpo',
  'Avery Bradley',
  'Devontae Cacok',
  'Kentavious Caldwell-Pope',
  'Alex Caruso',
  'Quinn Cook',
  'Troy Daniels',
  'Anthony Davis',
  'Jared Dudley',
  'Danny Green',
  'Talen Horton-Tucker',
  'Dwight Howard',
  'LeBron James',
  'Kyle Kuzma',
  'JaVale McGee',
  'Markieff Morris',
  'Zach Norvell',
  'Rajon Rondo',
  'J.R. Smith',
  'Dion Waiters'],
 'Page': ['https://www.basketball-reference.com//players/a/antetko01.html',
  'https://www.basketball-reference.com//players/b/bradlav01.html',
  'https://www.basketball-reference.com//players/c/cacokde01.html',
  'https://www.basketball-reference.com//players/c/caldwke01.html',
  'https://www.basketball-reference.com//players/c/carusal01.html',
  'https://www.basketball-reference.com//players/c/cookqu01.html',
  'https://www.basketball-reference.com//players/d/danietr01.html',
  'https://www.basketball-reference.com//players/d/davisan02.html',
  'https://www.basketball-reference.com//players/d/dud

In [4]:
for i in range(0, len(ballers['Page'])):
    
    # setup scrape
    url = ballers['Page'][i]
    html = requests.get(url)

    # scrape
    soup = bs(html.text, 'html.parser')
    table = soup.find_all(string=lambda text: isinstance(text, Comment))[46]

    stats = pd.read_html(table)[0]

    advanced = stats.loc[(stats['Season'] == '2019-20') | 
              (stats['Season'] == '2018-19') |
              (stats['Season'] == '2017-18') | 
              (stats['Season'] == '2016-17') | 
              (stats['Season'] == '2015-16') ].drop(['Age', 'Tm', 'Lg', 'Pos', 'G', 'MP'], axis=1)

    stats = pd.read_html(url)[0]

    standard = stats.loc[(stats['Season'] == '2019-20') | 
              (stats['Season'] == '2018-19') |
              (stats['Season'] == '2017-18') | 
              (stats['Season'] == '2016-17') | 
              (stats['Season'] == '2015-16') ]

    stats = pd.merge(standard, advanced, on='Season')
    
    if i == 0:
        for c in stats.columns:
                ballers[c] = []    

    for j in range(0, len(stats.columns)):
        stat = stats.columns[j]
        if j < 5:
            ballers[stat].append(stats[stat][len(stats[stat])-1])
        elif j < 7:
            ballers[stat].append(stats[stat].sum())
        else:
            ballers[stat].append(stats[stat].mean())
            
    ballers['Photo'].append(soup.find_all('img', alt=("Photo of " + ballers['Name'][i]))[0]['src'])

            

ballers.pop('Page')
ballers.pop('Season')
ballers.pop('Unnamed: 24')
ballers.pop('Unnamed: 19')
ballers


{'Name': ['Kostas Antetokounmpo',
  'Avery Bradley',
  'Devontae Cacok',
  'Kentavious Caldwell-Pope',
  'Alex Caruso',
  'Quinn Cook',
  'Troy Daniels',
  'Anthony Davis',
  'Jared Dudley',
  'Danny Green',
  'Talen Horton-Tucker',
  'Dwight Howard',
  'LeBron James',
  'Kyle Kuzma',
  'JaVale McGee',
  'Markieff Morris',
  'Zach Norvell',
  'Rajon Rondo',
  'J.R. Smith',
  'Dion Waiters'],
 'Photo': ['https://www.basketball-reference.com/req/202104203/images/players/antetko01.jpg',
  'https://www.basketball-reference.com/req/202104203/images/players/bradlav01.jpg',
  'https://www.basketball-reference.com/req/202104203/images/players/cacokde01.jpg',
  'https://www.basketball-reference.com/req/202104203/images/players/caldwke01.jpg',
  'https://www.basketball-reference.com/req/202104203/images/players/carusal01.jpg',
  'https://www.basketball-reference.com/req/202104203/images/players/cookqu01.jpg',
  'https://www.basketball-reference.com/req/202104203/images/players/danietr01.jpg',
  

In [5]:
baller_df = pd.DataFrame(ballers)
baller_df

,Name,Photo,Age,Tm,Lg,Pos,G,GS,MP,FG,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,Kostas Antetokounmpo,https://www.basketball-reference.com/req/20210...,22.0,LAL,NBA,PF,7.0,0.0,4.750000,0.300000,...,18.950000,16.400000,0.000000,0.000000,0.000000,-0.027000,-6.200000,-0.500000,-6.700000,0.000000
1,Avery Bradley,https://www.basketball-reference.com/req/20210...,29.0,LAL,NBA,SG,834.0,825.0,30.347619,4.919048,...,12.295238,19.576190,-0.442857,0.923810,0.490476,0.020381,-2.490476,-0.728571,-3.223810,-0.361905
2,Devontae Cacok,https://www.basketball-reference.com/req/20210...,23.0,LAL,NBA,C,1.0,0.0,9.000000,3.000000,...,0.000000,28.100000,0.000000,0.000000,0.000000,0.226000,4.800000,-4.600000,0.200000,0.000000
3,Kentavious Caldwell-Pope,https://www.basketball-reference.com/req/20210...,26.0,LAL,NBA,SG,377.0,274.0,30.700000,4.440000,...,8.840000,17.420000,2.280000,2.320000,4.560000,0.095600,-0.720000,0.420000,-0.300000,1.000000
4,Alex Caruso,https://www.basketball-reference.com/req/20210...,25.0,LAL,NBA,PG,126.0,13.0,18.266667,2.133333,...,18.533333,15.233333,0.300000,1.000000,1.266667,0.076333,-2.466667,1.333333,-1.100000,0.200000
5,Quinn Cook,https://www.basketball-reference.com/req/20210...,26.0,LAL,NBA,PG,235.0,29.0,14.291667,2.466667,...,15.366667,19.691667,0.208333,0.191667,0.408333,0.045500,-1.591667,-1.600000,-3.200000,-0.058333
6,Troy Daniels,https://www.basketball-reference.com/req/20210...,28.0,DEN,NBA,SG,522.0,19.0,13.038462,1.853846,...,7.600000,17.476923,0.253846,0.330769,0.615385,0.040231,-1.792308,-1.623077,-3.407692,-0.069231
7,Anthony Davis,https://www.basketball-reference.com/req/20210...,26.0,LAL,NBA,PF,329.0,329.0,35.080000,9.660000,...,8.980000,30.200000,6.440000,4.060000,10.500000,0.218600,5.160000,1.620000,6.780000,5.040000
8,Jared Dudley,https://www.basketball-reference.com/req/20210...,34.0,LAL,NBA,PF,297.0,74.0,18.060000,1.720000,...,14.420000,11.020000,1.020000,0.960000,1.960000,0.079800,-2.120000,0.880000,-1.240000,0.280000
9,Danny Green,https://www.basketball-reference.com/req/20210...,32.0,LAL,NBA,SG,365.0,355.0,26.160000,3.000000,...,11.680000,14.540000,0.900000,3.080000,3.960000,0.098200,-0.600000,1.540000,0.960000,1.460000


In [ ]:
# STOP HERE!!!!

In [41]:
len(baller_df['Name'])

20

In [44]:
baller_percentile = {}



for c in baller_df.columns:
            baller_percentile[c] = []    

        
        
        
        
for i in range(0, len(baller_df.columns)):
    stat = baller_df.columns[i]
    
    ranked = baller_df.sort_values(stat, ascending=False)[stat].to_numpy()
    
    
    
    for j in range(0, len(baller_df[stat])):
        
        for k in range(0, len(ranked)):
            if ranked[k] == baller_df[stat][j]:
                print([baller_df['Name'][j], stat, baller_df[stat][j], j])
    
    
#     baller_percentile[stat].append(baller_df[stat].max())
    
    
#     if j < 5:
#         ballers[stat].append(stats[stat][len(stats[stat])-1])
#     elif j < 7:
#         ballers[stat].append(stats[stat].sum())
#     else:
#         ballers[stat].append(stats[stat].mean())

baller_percentile


ranked = baller_df.sort_values('PTS', ascending=False)
# ['Name'].to_numpy()

ranked


        


['Kostas Antetokounmpo', 'Name', 'Kostas Antetokounmpo', 0]
['Avery Bradley', 'Name', 'Avery Bradley', 1]
['Devontae Cacok', 'Name', 'Devontae Cacok', 2]
['Kentavious Caldwell-Pope', 'Name', 'Kentavious Caldwell-Pope', 3]
['Alex Caruso', 'Name', 'Alex Caruso', 4]
['Quinn Cook', 'Name', 'Quinn Cook', 5]
['Troy Daniels', 'Name', 'Troy Daniels', 6]
['Anthony Davis', 'Name', 'Anthony Davis', 7]
['Jared Dudley', 'Name', 'Jared Dudley', 8]
['Danny Green', 'Name', 'Danny Green', 9]
['Talen Horton-Tucker', 'Name', 'Talen Horton-Tucker', 10]
['Dwight Howard', 'Name', 'Dwight Howard', 11]
['LeBron James', 'Name', 'LeBron James', 12]
['Kyle Kuzma', 'Name', 'Kyle Kuzma', 13]
['JaVale McGee', 'Name', 'JaVale McGee', 14]
['Markieff Morris', 'Name', 'Markieff Morris', 15]
['Zach Norvell', 'Name', 'Zach Norvell', 16]
['Rajon Rondo', 'Name', 'Rajon Rondo', 17]
['J.R. Smith', 'Name', 'J.R. Smith', 18]
['Dion Waiters', 'Name', 'Dion Waiters', 19]
['Kostas Antetokounmpo', 'Photo', 'https://www.basketball-

['Devontae Cacok', 'FTr', 0.0, 2]
['Kentavious Caldwell-Pope', 'FTr', 0.21799999999999997, 3]
['Alex Caruso', 'FTr', 0.3173333333333333, 4]
['Quinn Cook', 'FTr', 0.07583333333333332, 5]
['Troy Daniels', 'FTr', 0.07023076923076922, 6]
['Anthony Davis', 'FTr', 0.4226, 7]
['Jared Dudley', 'FTr', 0.18019999999999997, 8]
['Danny Green', 'FTr', 0.088, 9]
['Talen Horton-Tucker', 'FTr', 0.133, 10]
['Dwight Howard', 'FTr', 0.7180000000000001, 11]
['LeBron James', 'FTr', 0.3504, 12]
['Kyle Kuzma', 'FTr', 0.22066666666666668, 13]
['JaVale McGee', 'FTr', 0.2718, 14]
['Markieff Morris', 'FTr', 0.18244827586206894, 15]
['Zach Norvell', 'FTr', 0.058, 16]
['Rajon Rondo', 'FTr', 0.11640000000000002, 17]
['J.R. Smith', 'FTr', 0.08759999999999998, 18]
['Dion Waiters', 'FTr', 0.15084615384615382, 19]
['Kostas Antetokounmpo', 'ORB%', 5.55, 0]
['Avery Bradley', 'ORB%', 2.1380952380952385, 1]
['Devontae Cacok', 'ORB%', 24.6, 2]
['Kentavious Caldwell-Pope', 'ORB%', 2.48, 3]
['Alex Caruso', 'ORB%', 2.833333333

,Name,Photo,Age,Tm,Lg,Pos,G,GS,MP,FG,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
7,Anthony Davis,https://www.basketball-reference.com/req/20210...,26.0,LAL,NBA,PF,329.0,329.0,35.080000,9.660000,...,8.980000,30.200000,6.440000,4.060000,10.500000,0.218600,5.160000,1.620000,6.780000,5.040000
12,LeBron James,https://www.basketball-reference.com/req/20210...,35.0,LAL,NBA,PG,354.0,354.0,36.020000,9.960000,...,14.760000,31.220000,8.260000,3.240000,11.500000,0.213400,6.740000,1.620000,8.340000,6.680000
13,Kyle Kuzma,https://www.basketball-reference.com/req/20210...,24.0,LAL,NBA,PF,208.0,114.0,29.766667,6.000000,...,10.666667,23.066667,0.933333,2.033333,2.966667,0.068333,-0.500000,-1.066667,-1.566667,0.333333
11,Dwight Howard,https://www.basketball-reference.com/req/20210...,34.0,LAL,NBA,C,304.0,237.0,27.340000,4.860000,...,16.640000,19.220000,2.680000,2.860000,5.540000,0.154400,-0.340000,0.040000,-0.280000,0.980000
3,Kentavious Caldwell-Pope,https://www.basketball-reference.com/req/20210...,26.0,LAL,NBA,SG,377.0,274.0,30.700000,4.440000,...,8.840000,17.420000,2.280000,2.320000,4.560000,0.095600,-0.720000,0.420000,-0.300000,1.000000
1,Avery Bradley,https://www.basketball-reference.com/req/20210...,29.0,LAL,NBA,SG,834.0,825.0,30.347619,4.919048,...,12.295238,19.576190,-0.442857,0.923810,0.490476,0.020381,-2.490476,-0.728571,-3.223810,-0.361905
19,Dion Waiters,https://www.basketball-reference.com/req/20210...,28.0,LAL,NBA,SG,258.0,116.0,22.238462,4.230769,...,12.530769,25.369231,-0.076923,0.515385,0.430769,0.027231,-0.292308,-0.607692,-0.900000,0.015385
15,Markieff Morris,https://www.basketball-reference.com/req/20210...,30.0,LAL,NBA,PF,1229.0,617.0,22.479310,3.865517,...,11.600000,20.817241,0.086207,0.934483,0.989655,0.053828,-1.841379,-0.362069,-2.237931,-0.068966
17,Rajon Rondo,https://www.basketball-reference.com/req/20210...,33.0,LAL,NBA,PG,300.0,209.0,27.680000,3.700000,...,22.560000,17.920000,0.740000,1.900000,2.640000,0.072600,-0.700000,0.140000,-0.520000,0.740000
9,Danny Green,https://www.basketball-reference.com/req/20210...,32.0,LAL,NBA,SG,365.0,355.0,26.160000,3.000000,...,11.680000,14.540000,0.900000,3.080000,3.960000,0.098200,-0.600000,1.540000,0.960000,1.460000


In [ ]:
# for baller in ballers:
#     print(pd.read_html(baller)[0])

# setup scrape
url = ballers['Page'][0]

html = requests.get(url)

# scrape
soup = bs(html.text, 'html.parser')
table = soup.find_all(string=lambda text: isinstance(text, Comment))[46]

stats = pd.read_html(table)[0]

stats

advanced = stats.loc[(stats['Season'] == '2019-20') | 
          (stats['Season'] == '2018-19') |
          (stats['Season'] == '2017-18') | 
          (stats['Season'] == '2016-17') | 
          (stats['Season'] == '2015-16') ].drop(['Age', 'Tm', 'Lg', 'Pos', 'G', 'MP'], axis=1)

stats = pd.read_html(url)[0]

standard = stats.loc[(stats['Season'] == '2019-20') | 
          (stats['Season'] == '2018-19') |
          (stats['Season'] == '2017-18') | 
          (stats['Season'] == '2016-17') | 
          (stats['Season'] == '2015-16') ]

stats = pd.merge(standard, advanced, on='Season')

this = {}

this['Photo'] = soup.find_all('img', alt=("Photo of " + ballers['Name'][0]))[0]['src']

this




In [ ]:
kostas = {}

kostas['Name'] = ballers['Name'][0]

i = 0
for c in stats.columns:
    if i < 5:
        kostas[c] = stats[c][len(stats[c])-1]
    elif i < 7:
        kostas[c] = stats[c].sum()
    else:
        kostas[c] = stats[c].mean()
    i = i+1
    
kostas['Photo'] = ""

ballers['Name'][12]

# kostas.pop('Season')
# kostas.pop('Unnamed: 24')
# kostas.pop('Unnamed: 19')
# kostas

In [ ]:
# quit the browser
browser.quit()

In [ ]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(url)

# Jump to image link and setup soup
browser.links.find_by_partial_text('FULL IMAGE').click()
html = browser.html
soup = bs(html, 'html.parser')

# Get featured image url
featured_image_url = url.replace('index.html', soup.find_all('img')[1]['src'])

# quit the browser
browser.quit()


In [ ]:
# Try to scrape the site using the URL
url = 'https://www.basketball-reference.com/players/j/jamesle01.html'

# Retrieve page with the requests module
html = requests.get(url)

# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html.text, 'html.parser')

# Get article title and summary paragraph

lebron_face = soup.find_all('img', alt=("Photo of " + ballers['Name'][12]))[0]['src']
st = soup.find_all('table')[0]

st

lebron_face

# news_H = soup.find_all('div', class_='content_title')[0].a.text.replace("\n", "")
# news_P = soup.find_all('div', class_='rollover_description_inner')[0].text.replace("\n", "")


In [ ]:
# Try to scrape the site using the URL
url = 'https://space-facts.com/mars/'

# Retrieve page with the requests module
html = requests.get(url)

# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html.text, 'html.parser')

# Get Mars descriptions
col1 = soup.find_all('td', class_='column-1')
col2 = soup.find_all('td', class_='column-2')
props = []
stats = []

# Put descriptions in a dataframe
for x in range(0, len(col1)):
    props.append(col1[x].text)
    stats.append(col2[x].text)
    
fax_table = pd.DataFrame({"Description": props, "Mars": stats}).set_index('Description').to_html().replace("\n", "").replace("   ", " ")



In [ ]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

html = browser.html
soup = bs(html, 'html.parser')

# Get pages to jump to and set arrays to fill
jumps = []
titles = []
img_urls = []
hemisphere_urls = []

that = soup.find_all('h3')

for this in that:
    jumps.append(this.text)
    
# Jump through pages and get image titles and urls
for jump in jumps:
    try:
        browser.links.find_by_partial_text(jump).click()
        html = browser.html
        soup = bs(html, 'html.parser')
        titles.append(soup.find_all('h2')[0].text.replace(" Enhanced", ""))
        img_urls.append(soup.find_all('a')[4]['href'])
        print("1 " + jump)
    except:
        try:
            browser.links.find_by_partial_text('2').click()
            browser.links.find_by_partial_text(jump).click()
            html = browser.html
            soup = bs(html, 'html.parser')
            titles.append(soup.find_all('h2')[0].text.replace(" Enhanced", ""))
            img_urls.append(soup.find_all('a')[4]['href'])
            print("2 " + jump)
        except:
            browser.links.find_by_partial_text('1').click()
            browser.links.find_by_partial_text(jump).click()
            html = browser.html
            soup = bs(html, 'html.parser')
            titles.append(soup.find_all('h2')[0].text.replace(" Enhanced", ""))
            img_urls.append(soup.find_all('a')[4]['href'])
            print("1 " + jump)

# Quit the browser
browser.quit()

# Set results to list of dictionaries
for x in range(0, len(img_urls)):
    hemisphere_urls.append({"title": titles[x], "img_url": img_urls[x]})


In [ ]:
mars = {"news_H": news_H,
        "news_P": news_P,
        "featured_image_url": featured_image_url,
        "fax_table": fax_table,
        "hemisphere_urls": hemisphere_urls}
mars
